# Analytical calculations of power

- Formula:
  \begin{align*}
  \text{Power} &= \Phi\left(\frac{|\tau| \sqrt{N}}{2\sigma}- \Phi^{-1}(1- \frac{\alpha}{2})\right)
  \end{align*}

- Components:
  - $\Phi$: standard normal CDF is monotonically increasing
  - $\tau$: the effect size
  - $N$: the sample size
  - $\sigma$: the standard deviation of the outcome
  - $\alpha$: the significance level (typically 0.05)

In [1]:
require("pwr")

Loading required package: pwr



In [5]:
pwr.t.test(n = 2000, d = 0.1, sig.level = 0.05, power = NULL )


     Two-sample t test power calculation 

              n = 2000
              d = 0.1
      sig.level = 0.05
          power = 0.885232
    alternative = two.sided

NOTE: n is number in *each* group


### Effect Size: Minimum detectable effect

  \begin{align*}
  \tau &= \left( t_{1-\kappa} + t_{\alpha} \right) * \sqrt{\frac{1}{p(1-p)}}* \sqrt{\frac{\sigma^2}{N}}
  \end{align*}
  
  - $1-\kappa$ statistical power
  


## Sample size for a given minimum detectable effect size 

In [6]:
# https://github.com/vikjam/pwrcalc
#devtools::install_github('vikjam/pwrcalc')
require("pwrcalc")

Loading required package: pwrcalc

Loading required package: magrittr



In [14]:
power = 0.8                                                                     #SPECIFY - desired power
nratio = 2                                                                     #SPECIFY - the ratio of the size of the treatment group to control group
alpha =0.05                                                                     #SPECIFY - significance level
p = nratio/(1+nratio)
  
baseline_mean <-  0                              #Record the mean of the outcome variable at baseline   
baseline_sd <- 1                                      #Record the standard deviation of the outcome variable at baseline   
  
expected_effect = 0.1*baseline_sd                                             #The expected effect should be specified based on the intervention and the cost. 
                                                                                #Here it is 0.3 times the sd
treated_mean <- expected_effect + baseline_mean
  
base_model = twomeans(m1 = baseline_mean, m2 = treated_mean, sd = baseline_sd, 
                        nratio=nratio, power=power, sig.level = alpha)

base_model


     Two-sample t-test power calculation 

             m1 = 0
             m2 = 0.1
             n1 = 1178
             n2 = 2355
      sig.level = 0.05
          power = 0.8
    alternative = two.sided

NOTE: 
m1 and m2 are the means of group 1 and 2, respectively.
n1 and n2 are the obs. of group 1 and 2, respectively.


In [15]:
cat("We need a minimum treatment size of",base_model$n2,"and control size of", 
      base_model$n1, "to detect an effect of", 
      expected_effect, "with a probability of", 
      power,  "if the effect is true and the ratio of the treatment and control is",nratio)

We need a minimum treatment size of 2355 and control size of 1178 to detect an effect of 0.1 with a probability of 0.8 if the effect is true and the ratio of the treatment and control is 2


## Limitations to analytical power calculations

- Only derived for some test statistics (differences of means)

- Makes specific assumptions about the data-generating process

- Incompatible with more complex designs




# Simulation-based power calculation

- Create dataset and simulate research design.

- Assumptions are necessary for simulation studies, but you make your own.

- For the DeclareDesign approach, see <https://declaredesign.org/>

## Steps


  - Modelo: conjunto de modelos de qué causa qué y cómo
  
  - Pregunta: una pregunta formulada en términos del modelo.
  
  - Estrategia de datos: el conjunto de procedimientos que utilizamos para recopilar información del mundo (muestreo, asignación, medición)
     
  - Estrategia de respuesta: cómo resumimos los datos producidos por la estrategia de datos


In [16]:
require("DeclareDesign")

Loading required package: DeclareDesign

Loading required package: randomizr

Loading required package: fabricatr

Loading required package: estimatr



In [17]:
require("DesignLibrary")

Loading required package: DesignLibrary



In [19]:
#Model

design_two_arm<-DesignLibrary::two_arm_designer()

In [20]:
design_two_arm


Research design declaration summary

Step 1 (population): declare_population(N = N, u_0 = rnorm(N), u_1 = rnorm(n = N, mean = rho * u_0, sd = sqrt(1 - rho^2))) 

Step 2 (potential outcomes): declare_potential_outcomes(Y ~ (1 - Z) * (u_0 * control_sd + control_mean) + Z * (u_1 * treatment_sd + treatment_mean)) 

Step 3 (inquiry): declare_inquiry(ATE = mean(Y_Z_1 - Y_Z_0)) -------------------

Step 4 (assignment): declare_assignment(Z = complete_ra(N, prob = assignment_prob)) 

Step 5 (reveal): declare_reveal() ----------------------------------------------

Step 6 (estimator): declare_estimator(Y ~ Z, inquiry = estimand) ---------------

Run of the design:

 inquiry estimand estimator term estimate std.error statistic  p.value conf.low
     ATE        1 estimator    Z     1.03     0.216      4.79 6.02e-06    0.606
 conf.high df outcome
      1.46 98       Y


In [21]:
data_mundo1<-draw_data(design_two_arm)

In [23]:
head(data_mundo1)

,ID,u_0,u_1,Y_Z_0,Y_Z_1,Z,Y
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,001,-0.1225205,-0.1225205,-0.1225205,0.8774795,0,-0.1225205
2,002,-0.2535749,-0.2535749,-0.2535749,0.7464251,0,-0.2535749
3,003,1.2221940,1.2221940,1.2221940,2.2221940,1,2.2221940
4,004,1.1435621,1.1435621,1.1435621,2.1435621,0,1.1435621
5,005,-1.3045343,-1.3045343,-1.3045343,-0.3045343,1,-0.3045343
6,006,-1.2360175,-1.2360175,-1.2360175,-0.2360175,0,-1.2360175


In [24]:
head(draw_data(design_two_arm))

,ID,u_0,u_1,Y_Z_0,Y_Z_1,Z,Y
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,001,-1.4281171,-1.4281171,-1.4281171,-0.4281171,1,-0.4281171
2,002,0.3804937,0.3804937,0.3804937,1.3804937,0,0.3804937
3,003,-0.5880345,-0.5880345,-0.5880345,0.4119655,0,-0.5880345
4,004,1.0705576,1.0705576,1.0705576,2.0705576,1,2.0705576
5,005,-0.6999435,-0.6999435,-0.6999435,0.3000565,1,0.3000565
6,006,0.8617654,0.8617654,0.8617654,1.8617654,1,1.8617654


In [25]:
dim(data_mundo1)

[1] 100   7

In [26]:
table(data_mundo1$Z)


 0  1 
50 50 

In [27]:
lm(Y~Z,data=data_mundo1)


Call:
lm(formula = Y ~ Z, data = data_mundo1)

Coefficients:
(Intercept)            Z  
    -0.2264       1.0345  


In [28]:
draw_estimand(design_two_arm)

inquiry,estimand
<chr>,<dbl>
ATE,1


In [29]:
draw_estimates(design_two_arm)

estimator,term,estimate,std.error,statistic,p.value,conf.low,conf.high,df,outcome,inquiry
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
estimator,Z,1.148202,0.2231321,5.145842,1.370055e-06,0.705404,1.591001,98,Y,ATE


In [30]:
simulate_design(design_two_arm,sims=3)

Warning message:
“We recommend you choose a number of simulations higher than 30.”


design,sim_ID,inquiry,estimand,estimator,term,estimate,std.error,statistic,p.value,conf.low,conf.high,df,outcome
<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
design_two_arm,1,ATE,1,estimator,Z,0.7701154,0.1902542,4.047823,1.033323e-04,0.3925621,1.147669,98,Y
design_two_arm,2,ATE,1,estimator,Z,0.9095218,0.2236282,4.067116,9.628426e-05,0.4657389,1.353305,98,Y
design_two_arm,3,ATE,1,estimator,Z,1.4859902,0.2041970,7.277237,8.532577e-11,1.0807678,1.891213,98,Y


In [31]:
diagnose_design(design_two_arm,sims=100)


Research design diagnosis based on 100 simulations. Diagnosis completed in 1 secs. Diagnosand estimates with bootstrapped standard errors in parentheses (100 replicates).

         Design Inquiry Estimator Outcome Term N Sims Mean Estimand
 design_two_arm     ATE estimator       Y    Z    100          1.00
                                                             (0.00)
 Mean Estimate   Bias SD Estimate   RMSE  Power Coverage
          1.00   0.00        0.17   0.17   1.00     0.98
        (0.02) (0.02)      (0.01) (0.01) (0.00)   (0.01)